# SIFT from Scratch - Analysis Notebook

This notebook demonstrates the SIFT implementation and runs various experiments:
- Keypoint detection and visualization
- Descriptor matching
- Rotation and scale robustness tests
- Comparison with OpenCV SIFT

In [ ]:
# Import libraries
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.sift_from_scratch import SIFT_Scratch, match_descriptors, draw_matches
from src.utils import read_color, rotate_image, resize_image, draw_keypoints_custom

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Test Images

In [ ]:
# Load image pairs
img1 = cv2.imread('../data/graffiti/img1.jpg')
img2 = cv2.imread('../data/graffiti/img2.jpg')

if img1 is None or img2 is None:
    print("Images not found. Please place images in data/graffiti/")
else:
    print(f"Image 1 shape: {img1.shape}")
    print(f"Image 2 shape: {img2.shape}")
    
    # Display images
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    axes[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Image 1')
    axes[0].axis('off')
    axes[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Image 2')
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()

## 2. SIFT Detection and Description

In [ ]:
# Create SIFT detector
sift = SIFT_Scratch(num_octaves=3, scales_per_octave=3, 
                   contrast_thresh=0.04, edge_thresh=10)

print("Detecting keypoints...")
kp1, desc1 = sift.detect_and_compute(img1)
kp2, desc2 = sift.detect_and_compute(img2)

print(f"Keypoints in image 1: {len(kp1)}")
print(f"Keypoints in image 2: {len(kp2)}")
print(f"Descriptor shape: {desc1.shape if len(desc1) > 0 else 'N/A'}")

## 3. Visualize Keypoints and Match Descriptors

In [ ]:
# Match descriptors with ratio test
matches = match_descriptors(desc1, desc2, ratio=0.75)
print(f"Number of matches: {len(matches)}")
print(f"Match ratio: {len(matches) / max(len(kp1), len(kp2)):.3f}")

# Visualize matches
img_matches = draw_matches(img1, kp1, img2, kp2, matches, max_matches=100)
plt.figure(figsize=(16, 8))
plt.imshow(cv2.cvtColor(img_matches, cv2.COLOR_BGR2RGB))
plt.title(f'SIFT Matches: {len(matches)}')
plt.axis('off')
plt.show()

## 4. Rotation Robustness Test

In [ ]:
# Test rotation invariance
angles = [0, 15, 30, 45, 60, 90]
rotation_results = []

for angle in angles:
    img_rot = rotate_image(img1, angle)
    kp_rot, desc_rot = sift.detect_and_compute(img_rot)
    matches_rot = match_descriptors(desc1, desc_rot, ratio=0.75)
    rotation_results.append(len(matches_rot))
    print(f"Rotation {angle:3d}°: {len(matches_rot)} matches")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(angles, rotation_results, 'o-', linewidth=2, markersize=8)
plt.xlabel('Rotation Angle (degrees)', fontsize=12)
plt.ylabel('Number of Matches', fontsize=12)
plt.title('Rotation Robustness', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()